In [51]:
emotions_list = ['admiration',
 'amusement',
 'anger',
 'annoyance',
 'approval',
 'caring',
 'confusion',
 'curiosity',
 'desire',
 'disappointment',
 'disapproval',
 'disgust',
 'embarrassment',
 'excitement',
 'fear',
 'gratitude',
 'grief',
 'joy',
 'love',
 'nervousness',
 'optimism',
 'pride',
 'realization',
 'relief',
 'remorse',
 'sadness',
 'surprise',
 'neutral']

ekman_mapping = {
"anger": ["anger", "annoyance", "disapproval"],
"disgust": ["disgust"],
"fear": ["fear", "nervousness"],
"joy": ["joy", "amusement", "approval", "excitement", "gratitude",  "love", "optimism", "relief", "pride", "admiration", "desire", "caring"],
"sadness": ["sadness", "disappointment", "embarrassment", "grief",  "remorse"],
"surprise": ["surprise", "realization", "confusion", "curiosity"],
 "neutral": ["neutral"]  # my addition
}

ekman_list = list(ekman_mapping.keys())

sentiment_mapping = {
"positive": ["amusement", "excitement", "joy", "love", "desire", "optimism", "caring", "pride", "admiration", "gratitude", "relief", "approval"],
"negative": ["fear", "nervousness", "remorse", "embarrassment", "disappointment", "sadness", "grief", "disgust", "anger", "annoyance", "disapproval"],
"ambiguous": ["realization", "surprise", "curiosity", "confusion"],
 "neutral": ["neutral"] # my addition TODO deal with
}

sent_list = list(sentiment_mapping.keys())


vad_scaled_3 = [[0.93309693, 0.40324324, 0.74888093],
 [0.85298373, 0.8529148 , 0.85124654],
 [0.35      , 0.93645833, 0.66190476],
 [0.35      , 0.7029148 , 0.25428571],
 [0.65961538, 0.1496557 , 1.        ],
 [0.51681894, 0.17522124, 0.50763666],
 [0.40360139, 0.53629032, 0.11928571],
 [0.5825603 , 0.74439462, 0.42940171],
 [0.8       , 0.6559322 , 0.64216524],
 [0.21276596, 0.22282609, 0.23584906],
 [0.14673913, 0.34102564, 0.33333333],
 [0.        , 0.76689189, 0.1875    ],
 [0.26277372, 0.60860215, 0.04609665],
 [0.8       , 0.59917355, 0.75451264],
 [0.10326087, 0.86636771, 0.14785714],
 [0.74074074, 0.13511859, 0.5942813 ],
 [0.0923913 , 0.46370968, 0.44820513],
 [0.95807128, 0.83067293, 0.83733333],
 [1.        , 0.29069767, 0.70143241],
 [0.29927007, 1.        , 0.0639485 ],
 [0.88915009, 0.36742081, 0.88171745],
 [0.56307978, 0.44918919, 0.95464602],
 [0.48203991, 0.26453488, 0.93318966],
 [0.64659232, 0.04820513, 0.46925566],
 [0.1890411 , 0.55371901, 0.35370121],
 [0.        , 0.05286092, 0.        ],
 [0.70707071, 0.95402299, 0.55718702],
 [0.44739104, 0.        , 0.29714286]]

In [44]:
# convert to better data-structures
def get_key_of(mapping: dict[str, list[str]], val):
 """ Finds the key which `val` is an element in its list """
 for key, lst in mapping.items():
  if val in lst:
   return key
 return None

# idx --> group
ekman_mapping_rev = []
sentiment_mapping_rev = []

for emotion in emotions_list:
 ekman_mapping_rev.append(get_key_of(ekman_mapping, emotion))
 sentiment_mapping_rev.append(get_key_of(sentiment_mapping, emotion))

# ekman_mapping_rev

In [46]:
# read evaluated data
import numpy as np

path_eval_labels= r"C:\Users\MatanBT\My Drive\University\Year 3\Semester A\Natural Language Processing\NLP-Real-Project\emotion-recognition-nlp-project\paper-stuff\cahced-models\MAE_SCALED_checkpoint-20000\eval_labels.csv"

path_eval_preds = r"C:\Users\MatanBT\My Drive\University\Year 3\Semester A\Natural Language Processing\NLP-Real-Project\emotion-recognition-nlp-project\paper-stuff\cahced-models\MAE_SCALED_checkpoint-20000\eval_preds.csv"

emo_lbl_idx_to_vad = vad_scaled_3

eval_labels = np.loadtxt(path_eval_labels).astype(int)
eval_preds = np.loadtxt(path_eval_preds)

In [55]:
# 1NN
from sklearn.neighbors import NearestNeighbors
from datasets import tqdm

metric = 'manhattan'  # manhattan # TODO try: 'euclidean'
neigh = NearestNeighbors(n_neighbors=5, metric=metric)
neigh.fit(emo_lbl_idx_to_vad)

acc = np.zeros(len(eval_preds))
top_3_acc = np.zeros(len(eval_preds)) # allows the true label to be in the TOP3 rather than in TOP1
ekman_acc = np.zeros(len(eval_preds))
sent_acc = np.zeros(len(eval_preds))

eval_emotions = np.zeros((len(eval_preds), 2))
eval_ekman = np.zeros((len(eval_preds), 2))  # col-label, col-result
eval_sent = np.zeros((len(eval_preds), 2))   # col-label, col-result

for i, vad_pred in tqdm(enumerate(eval_preds), desc="Evaluating"):
 distances, labels = neigh.kneighbors(np.expand_dims(vad_pred, axis=0),
                                      return_distance=True)
 # distances = 1 - (distances / 2 ** 0.5)  # scaler to probability

 eval_emotions[i] = [eval_labels[i], labels[0, 0]]
 acc[i] = (eval_labels[i] == labels[0, 0])

 top_3_acc[i] = (eval_labels[i] in labels[0, 0:3])

 eval_ekman[i] = [ekman_list.index(ekman_mapping_rev[eval_labels[i]]),
                  ekman_list.index(ekman_mapping_rev[labels[0, 0]])]
 ekman_acc[i] = (eval_ekman[i][0] == eval_ekman[i][1])

 eval_sent[i] = [sent_list.index(sentiment_mapping_rev[eval_labels[i]]),
              sent_list.index(sentiment_mapping_rev[labels[0, 0]])]
 sent_acc[i] = (eval_sent[i][0] == eval_sent[i][1])

Evaluating: 0it [00:00, ?it/s]

>> accuracy: 0.5844327176781002


In [57]:
from sklearn.metrics import precision_recall_fscore_support

# Calc scores:
print(f"**** Emotions Scores [Overall] ****")
print(f">> accuracy: {acc.mean()}")
print(f">> accuracy (top-3): {top_3_acc.mean()}")
print(f">> precision, recall, macro-f1: {precision_recall_fscore_support(eval_emotions[:, 0], eval_emotions[:, 1], average='macro')}")

print(f"**** Ekman Scores [Overall]  ****")
print(f">> accuracy: {ekman_acc.mean()}")
print(f">> precision, recall, macro-f1: {precision_recall_fscore_support(eval_ekman[:, 0], eval_ekman[:, 1], average='macro')}")

print(f"**** Sentiment Scores [Overall]  ****")
print(f">> accuracy: {sent_acc.mean()}")
print(f">> precision, recall, macro-f1: {precision_recall_fscore_support(eval_sent[:, 0], eval_sent[:, 1], average='macro')}")

**** Emotions Scores ****
>> accuracy: 0.5844327176781002
>> accuracy (top-3): 0.6358839050131926
>> precision, recall, macro-f1: (0.4723648609384939, 0.4450570088465275, 0.4517017610183432, None)
**** Ekman Scores ****
>> accuracy: 0.6715039577836411
>> precision, recall, macro-f1: (0.603631220744588, 0.5715711896630478, 0.5860706762329626, None)
**** Sentiment Scores ****
>> accuracy: 0.6921723834652594
>> precision, recall, macro-f1: (0.6571358747243233, 0.648803506220825, 0.6527193488738827, None)


c:\users\matanbt\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [66]:
# Evaluate emotions per emotion
from sklearn.metrics import accuracy_score

for idx, emotion in enumerate(emotions_list):
 eval_emotion_only = (eval_emotions == idx).astype(int)
 print(f">>> Emotion `{emotion}`")
 print(f"   >> accuracy: {accuracy_score(eval_emotion_only[:, 0], eval_emotion_only[:, 1])}")
 print(f"   >> precision, recall, F1: {precision_recall_fscore_support(eval_emotion_only[:, 0], eval_emotion_only[:, 1], average='binary')}")

>>> Emotion `admiration`
   >> accuracy: 0.9575637642919965
   >> precision, recall, macro-f1: (0.6811989100817438, 0.7668711656441718, 0.7215007215007213, None)
>>> Emotion `amusement`
   >> accuracy: 0.9773526824978013
   >> precision, recall, macro-f1: (0.7091633466135459, 0.8557692307692307, 0.775599128540305, None)
>>> Emotion `anger`
   >> accuracy: 0.9797713280562885
   >> precision, recall, macro-f1: (0.6, 0.46788990825688076, 0.5257731958762887, None)
>>> Emotion `annoyance`
   >> accuracy: 0.9397537379067722
   >> precision, recall, macro-f1: (0.2544642857142857, 0.3475609756097561, 0.2938144329896907, None)
>>> Emotion `approval`
   >> accuracy: 0.940193491644679
   >> precision, recall, macro-f1: (0.4588235294117647, 0.3023255813953488, 0.36448598130841114, None)
>>> Emotion `caring`
   >> accuracy: 0.9659190853122251
   >> precision, recall, macro-f1: (0.2748091603053435, 0.375, 0.31718061674008813, None)
>>> Emotion `confusion`
   >> accuracy: 0.969217238346526
   >> prec

c:\users\matanbt\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


   >> precision, recall, macro-f1: (0.7, 0.8901734104046243, 0.7837150127226463, None)
>>> Emotion `nervousness`
   >> accuracy: 0.9975813544415127
   >> precision, recall, macro-f1: (0.3333333333333333, 0.375, 0.35294117647058826, None)
>>> Emotion `optimism`
   >> accuracy: 0.9795514511873351
   >> precision, recall, macro-f1: (0.6382978723404256, 0.5042016806722689, 0.5633802816901409, None)
>>> Emotion `pride`
   >> accuracy: 0.9978012313104662
   >> precision, recall, macro-f1: (0.0, 0.0, 0.0, None)
>>> Emotion `realization`
   >> accuracy: 0.9819700967458224
   >> precision, recall, macro-f1: (0.4, 0.21621621621621623, 0.2807017543859649, None)
>>> Emotion `relief`
   >> accuracy: 0.9958223394898856
   >> precision, recall, macro-f1: (0.0, 0.0, 0.0, None)
>>> Emotion `remorse`
   >> accuracy: 0.9940633245382586
   >> precision, recall, macro-f1: (0.6511627906976745, 0.7, 0.674698795180723, None)
>>> Emotion `sadness`
   >> accuracy: 0.9843887423043096
   >> precision, recall, mac

In [69]:
# Evaluate Ekman per category
from sklearn.metrics import accuracy_score

for idx, category in enumerate(ekman_list):
 eval_emotion_only = (eval_ekman == idx).astype(int)
 print(f">>> Category `{category}`")
 print(f"   >> accuracy: {accuracy_score(eval_emotion_only[:, 0], eval_emotion_only[:, 1])}")
 print(f"   >> precision, recall, F1: {precision_recall_fscore_support(eval_emotion_only[:, 0], eval_emotion_only[:, 1], average='binary')}")

>>> Category `anger`
   >> accuracy: 0.8975373790677221
   >> precision, recall, F1: (0.5206971677559913, 0.4927835051546392, 0.5063559322033898, None)
>>> Category `disgust`
   >> accuracy: 0.9859278803869833
   >> precision, recall, F1: (0.46938775510204084, 0.3770491803278688, 0.41818181818181815, None)
>>> Category `fear`
   >> accuracy: 0.9916446789797714
   >> precision, recall, F1: (0.75, 0.6363636363636364, 0.6885245901639345, None)
>>> Category `joy`
   >> accuracy: 0.8518029903254177
   >> precision, recall, F1: (0.7958333333333333, 0.8015587529976019, 0.7986857825567503, None)
>>> Category `sadness`
   >> accuracy: 0.947009674582234
   >> precision, recall, F1: (0.5, 0.5103734439834025, 0.5051334702258726, None)
>>> Category `surprise`
   >> accuracy: 0.9118293755496921
   >> precision, recall, F1: (0.5412621359223301, 0.5126436781609195, 0.5265643447461629, None)
>>> Category `neutral`
   >> accuracy: 0.7572559366754618
   >> precision, recall, F1: (0.6482381530984204, 0.67

In [68]:
# Evaluate Sentiment per sentiment
from sklearn.metrics import accuracy_score

for idx, sent in enumerate(sent_list):
 eval_emotion_only = (eval_sent == idx).astype(int)
 print(f">>> Sentiment: `{sent}`")
 print(f"   >> accuracy: {accuracy_score(eval_emotion_only[:, 0], eval_emotion_only[:, 1])}")
 print(f"   >> precision, recall, F1: {precision_recall_fscore_support(eval_emotion_only[:, 0], eval_emotion_only[:, 1], average='binary')}")

>>> Sentiment: `positive`
   >> accuracy: 0.8518029903254177
   >> precision, recall, F1: (0.7958333333333333, 0.8015587529976019, 0.7986857825567503, None)
>>> Sentiment: `negative`
   >> accuracy: 0.8634564643799473
   >> precision, recall, F1: (0.6432098765432098, 0.6107854630715123, 0.6265784726398075, None)
>>> Sentiment: `ambiguous`
   >> accuracy: 0.9118293755496921
   >> precision, recall, F1: (0.5412621359223301, 0.5126436781609195, 0.5265643447461629, None)
>>> Sentiment: `neutral`
   >> accuracy: 0.7572559366754618
   >> precision, recall, F1: (0.6482381530984204, 0.6702261306532663, 0.6590487955528104, None)
